# DATA CLEAN UP 

## Clean up for the maindata set : data/tmdb_5000_movies.csv' will be comprised of : 
   - x
   - x
   - x
   - x





In [25]:
# Import required tools for clean up 
import pandas as pd
import json

In [26]:
# Read the first CSV file
df1 = pd.read_csv('data/tmdb_5000_movies.csv')


# Print the first few rows of each DataFrame
print("DataFrame 1:")
print(df1.head())


DataFrame 1:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"i

Drop non-relevant columns and drop columns being parsed out into separate new dataframes

In [27]:
# Drop the specified columns
df1 = df1.drop(columns=['homepage', 'original_title', 'overview', 'spoken_languages'])

# Drop the fields that have been parsed out to new DataFrames (genres, keywords, production_companies)
movie_metadata_df = df1.drop(columns=['genres', 'keywords', 'production_companies'])

In [28]:
# Create a mapping DataFrame for languages movie were created in for acronymn and full name 
language_mapping_df = pd.DataFrame({
    'language': ['en', 'de', 'es', 'zh', 'ja', 'fr', 'da', 'it', 'sv', 'hi',
                 'ru', 'pt', 'ko', 'af', 'ro', 'nl', 'ar', 'he', 'th', 'cn',
                 'tr', 'cs', 'fa', 'no', 'ps', 'vi', 'el', 'hu', 'nb', 'xx',
                 'id', 'pl', 'is', 'te', 'ta', 'ky', 'sl'],
    'full_original_name': ['English', 'German', 'Spanish', 'Chinese', 'Japanese', 'French',
                           'Danish', 'Italian', 'Swedish', 'Hindi', 'Russian', 'Portuguese',
                           'Korean', 'Afrikaans', 'Romanian', 'Dutch', 'Arabic', 'Hebrew',
                           'Thai', 'Chinese', 'Turkish', 'Czech', 'Persian', 'Norwegian',
                           'Pashto', 'Vietnamese', 'Greek', 'Hungarian', 'Norwegian Bokmål',
                           'Unknown', 'Indonesian', 'Polish', 'Icelandic', 'Telugu', 
                           'Tamil', 'Kyrgyz', 'Slovenian']
})

# Merge the mapping DataFrame with your existing DataFrame
movie_metadata_df = movie_metadata_df.merge(
    language_mapping_df,
    left_on='original_language',  # Column with the language acronym
    right_on='language',          # Column in the mapping DataFrame
    how='left'
)

# Display the updated DataFrame, including the new column
print(movie_metadata_df[['original_language', 'full_original_name']])


     original_language full_original_name
0                   en            English
1                   en            English
2                   en            English
3                   en            English
4                   en            English
...                ...                ...
4798                es            Spanish
4799                en            English
4800                en            English
4801                en            English
4802                en            English

[4803 rows x 2 columns]


In [29]:
# Function to parse the first movie production country

def parse_first_country(row):
    try:
        # Load the JSON string from the 'production_countries' column
        countries = json.loads(row['production_countries'])
        if countries:
            # Extract the first country's iso_3166_1 and name
            return countries[0]['iso_3166_1'], countries[0]['name']
        else:
            return None, None
    except (json.JSONDecodeError, IndexError):
        return None, None

# Apply the function to the DataFrame and create new columns
movie_metadata_df[['iso_3166_1', 'country_name']] = movie_metadata_df.apply(parse_first_country, axis=1, result_type='expand')

# Display the values only (without labels)
for index, row in movie_metadata_df.iterrows():
    iso_code = row['iso_3166_1']
    country = row['country_name']
    if iso_code and country:  # Only print if both values are present
        print(f"{iso_code}, {country}")


US, United States of America
US, United States of America
GB, United Kingdom
US, United States of America
US, United States of America
US, United States of America
US, United States of America
US, United States of America
GB, United Kingdom
US, United States of America
US, United States of America
GB, United Kingdom
JM, Jamaica
US, United States of America
GB, United Kingdom
CZ, Czech Republic
US, United States of America
US, United States of America
US, United States of America
NZ, New Zealand
US, United States of America
GB, United Kingdom
NZ, New Zealand
GB, United Kingdom
NZ, New Zealand
US, United States of America
US, United States of America
US, United States of America
US, United States of America
GB, United Kingdom
US, United States of America
CN, China
US, United States of America
CA, Canada
US, United States of America
US, United States of America
US, United States of America
US, United States of America
US, United States of America
US, United States of America
US, United St

In [30]:
# Convert the 'release_date' column to datetime
movie_metadata_df['release_date'] = pd.to_datetime(movie_metadata_df['release_date'])

# Function to determine the season and holiday for a govem movie release date
def get_season_and_holiday(date):
    if pd.isna(date):  # Check if the date is NaT
        return None, None

    # Define common US holidays
    holidays = {
        "01-01": "New Year's Day",
        "07-04": "Independence Day",
        "12-25": "Christmas",
        "11-11": "Veterans Day",
        "11-26": "Thanksgiving",
        "12-31": "New Year's Eve",
        "01-20": "Martin Luther King Jr. Day",
        "02-14": "Valentine's Day",
        "04-01": "April Fool's Day",
        "10-31": "Halloween"
    }

    # Check for holiday
    holiday_name = holidays.get(date.strftime("%m-%d"), None)
    
    # Determine the season
    if date.month in [12, 1, 2]:
        season = 'Winter'
    elif date.month in [3, 4, 5]:
        season = 'Spring'
    elif date.month in [6, 7, 8]:
        season = 'Summer'
    elif date.month in [9, 10, 11]:
        season = 'Fall'
    else:
        season = None

    return season, holiday_name

# Apply the function to create new columns
movie_metadata_df[['release_date_seasonholiday', 'holiday_name']] = movie_metadata_df['release_date'].apply(
    get_season_and_holiday).apply(pd.Series)

# Filter the DataFrame to show only rows with holidays
holiday_results = movie_metadata_df[movie_metadata_df['holiday_name'].notna()]

# Display the updated DataFrame with holiday results
print(holiday_results[['title', 'release_date', 'release_date_seasonholiday', 'holiday_name']])

# For debugging, also print the full DataFrame
print("\nFull DataFrame for reference:")
print(movie_metadata_df[['title', 'release_date', 'release_date_seasonholiday', 'holiday_name']])


                                  title release_date  \
44                            Furious 7   2015-04-01   
98    The Hobbit: An Unexpected Journey   2012-11-26   
176                        The Revenant   2015-12-25   
206                 Clash of the Titans   2010-04-01   
251                  Gulliver's Travels   2010-12-25   
...                                 ...          ...   
4774                     Pink Narcissus   1971-01-01   
4781                          Dry Spell   2013-02-14   
4782                           Flywheel   2003-01-01   
4791                        Tin Can Man   2007-01-01   
4794       Sanctuary: Quite a Conundrum   2012-01-20   

     release_date_seasonholiday                holiday_name  
44                       Spring            April Fool's Day  
98                         Fall                Thanksgiving  
176                      Winter                   Christmas  
206                      Spring            April Fool's Day  
251              

Format currency related columns into dollar format, vote count to thousands format, popularity score to single decimal point, and filter dataset where status = 'Released'

In [31]:
# Format the 'budget' and 'revenue' fields as currency using string formatting
movie_metadata_df['budget'] = movie_metadata_df['budget'].apply(lambda x: f"${x:,.0f}" if pd.notnull(x) else x)
movie_metadata_df['revenue'] = movie_metadata_df['revenue'].apply(lambda x: f"${x:,.0f}" if pd.notnull(x) else x)

# Format the 'vote_count' field to thousands (K) format
movie_metadata_df['vote_count'] = movie_metadata_df['vote_count'].apply(lambda x: f"{x/1000:.1f}K" if pd.notnull(x) else x)

# Format the 'popularity' field to one decimal point
movie_metadata_df['popularity'] = movie_metadata_df['popularity'].apply(lambda x: f"{x:.1f}" if pd.notnull(x) else x)

# Filter the DataFrame where 'status' is 'Released' and keep the name 'movie_metadata_df'
movie_metadata_df = movie_metadata_df[movie_metadata_df['status'] == 'Released']

Parse out ID and Name from genre, keywords, production company fields in dataset into column format and create separate dataframes. Keep movie_ID as primary key to join to other dataframes. Drop any records with missing data.

In [32]:
# Function to parse the 'genres' column and create separate rows for each genre
def parse_genres(row):
    try:
        # Parse the JSON string from the 'genres' column
        genres = json.loads(row['genres'])
        # Create a list of tuples (movie_id, genre_id, genre_name)
        return [(row['id'], genre['id'], genre['name']) for genre in genres]
    except json.JSONDecodeError:
        return []

# Function to parse the 'keywords' column and create separate rows for each keyword
def parse_keywords(row):
    try:
        # Parse the JSON string from the 'keywords' column
        keywords = json.loads(row['keywords'])
        # Create a list of tuples (movie_id, keyword_id, keyword_name)
        return [(row['id'], keyword['id'], keyword['name']) for keyword in keywords]
    except json.JSONDecodeError:
        return []

# Function to parse the 'production_companies' column and create separate rows for each company
def parse_production_companies(row):
    try:
        # Parse the JSON string from the 'production_companies' column
        companies = json.loads(row['production_companies'])
        # Create a list of tuples (movie_id, company_id, company_name)
        return [(row['id'], company['id'], company['name']) for company in companies]
    except json.JSONDecodeError:
        return []

# Apply the parsing functions to the 'genres', 'keywords', and 'production_companies' columns
genres_list = df1.apply(parse_genres, axis=1).explode()
keywords_list = df1.apply(parse_keywords, axis=1).explode()
companies_list = df1.apply(parse_production_companies, axis=1).explode()

# Create new DataFrames from the parsed lists
genre_df = pd.DataFrame(genres_list.tolist(), columns=['movie_id', 'genre_id', 'genre_name'])
keyword_df = pd.DataFrame(keywords_list.tolist(), columns=['movie_id', 'keyword_id', 'keyword_name'])
production_company_df = pd.DataFrame(companies_list.tolist(), columns=['movie_id', 'company_id', 'company_name'])

# Drop any rows with missing data (in case of malformed or missing information)
genre_df.dropna(inplace=True)
keyword_df.dropna(inplace=True)
production_company_df.dropna(inplace=True)

# Display the first few rows of the new DataFrames
print("Genres DataFrame:")
print(genre_df.head())

print("\nKeywords DataFrame:")
print(keyword_df.head())

print("\nProduction Companies DataFrame:")
print(production_company_df.head())

print("Movie Metadata DataFrame:")
print(movie_metadata_df.head())

Genres DataFrame:
   movie_id  genre_id       genre_name
0   19995.0      28.0           Action
1   19995.0      12.0        Adventure
2   19995.0      14.0          Fantasy
3   19995.0     878.0  Science Fiction
4     285.0      12.0        Adventure

Keywords DataFrame:
   movie_id  keyword_id   keyword_name
0   19995.0      1463.0  culture clash
1   19995.0      2964.0         future
2   19995.0      3386.0      space war
3   19995.0      3388.0   space colony
4   19995.0      3679.0        society

Production Companies DataFrame:
   movie_id  company_id                            company_name
0   19995.0       289.0                 Ingenious Film Partners
1   19995.0       306.0  Twentieth Century Fox Film Corporation
2   19995.0       444.0                      Dune Entertainment
3   19995.0       574.0                Lightstorm Entertainment
4     285.0         2.0                    Walt Disney Pictures
Movie Metadata DataFrame:
         budget      id original_language populari

In [33]:
# Define the file path based on your working directory
file_path = 'data/oscars_df.csv'

# Load the CSV file into a new DataFrame
oscars_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(oscars_df.head())

   Unnamed: 0                 Film Oscar Year Film Studio/Producer(s)  \
0           0                Wings    1927/28    Famous Players-Lasky   
1           1           7th Heaven    1927/28                     Fox   
2           2           The Racket    1927/28       The Caddo Company   
3           3  The Broadway Melody    1928/29     Metro-Goldwyn-Mayer   
4           4                Alibi    1928/29     Feature Productions   

     Award  Year of Release  Movie Time            Movie Genre  IMDB Rating  \
0   Winner             1927         144      Drama,Romance,War          7.5   
1  Nominee             1927         110          Drama,Romance          7.7   
2  Nominee             1928          84  Crime,Drama,Film-Noir          6.7   
3   Winner             1929         100  Drama,Musical,Romance          5.7   
4  Nominee             1929          91   Action,Crime,Romance          5.8   

  IMDB Votes  ... Tomatometer Status Tomatometer Rating Tomatometer Count  \
0     12,

Merge the oscar dataset with the movies metadata dataframe. Add a new column in the movie metadata dataframe called Award.

In [34]:
# Perform the merge/join operation, matching 'Film' from oscar_df with 'title' from movie_metadata_df
# Only the 'award' column from oscar_df will be added to movie_metadata_df
movie_metadata_df = movie_metadata_df.merge(oscars_df[['Film', 'Oscar Year', 'Year of Release', 'Award']], 
                                            how='left', 
                                            left_on='title', 
                                            right_on='Film')

# Drop the 'Film' column from the merge (since it's the same as 'title')
movie_metadata_df = movie_metadata_df.drop(columns=['Film'])

# Ensure 'award' is the last column
award_column = movie_metadata_df.pop('Award')
movie_metadata_df['Award'] = award_column

# Display the first few rows of the updated DataFrame
print(movie_metadata_df.head())

         budget      id original_language popularity  \
0  $237,000,000   19995                en      150.4   
1  $300,000,000     285                en      139.1   
2  $245,000,000  206647                en      107.4   
3  $250,000,000   49026                en      112.3   
4  $260,000,000   49529                en       43.9   

                                production_countries release_date  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   

          revenue  runtime    status  \
0  $2,787,965,087    162.0  Released   
1    $961,000,000    169.0  Released   
2    $880,674,609    148.0  Released   
3  $1,084,939,099    165.0  Released   
4    $284,139,100    132.0  Released   

       

Load the credits parquet file. Parse out the cast field with only the first ID and name in the string and create new columns along with the movie_id and title columns. Create a new dataframe. 

Roses version with added parsing cript for "crew"

In [35]:
import pandas as pd
import json

# Load the parquet file 
file_path = 'data/credits_5000.parquet'
credits_df = pd.read_parquet(file_path)

# Check if the 'crew' column exists
if 'crew' not in credits_df.columns or 'cast' not in credits_df.columns:
    raise KeyError("The required columns 'crew' or 'cast' are missing from the DataFrame.")

# Function to parse the 'cast' column and extract only the first 'id' and 'name'
def parse_first_cast(row):
    try:
        # Parse the JSON string from the 'cast' column
        cast_list = json.loads(row['cast'])
        if cast_list:
            # Return the first cast member's id and name
            return (row['movie_id'], row['title'], cast_list[0]['id'], cast_list[0]['name'])
        else:
            return (row['movie_id'], row['title'], None, None)
    except (json.JSONDecodeError, IndexError):
        return (row['movie_id'], row['title'], None, None)

# Function to parse the 'crew' column and extract only the first 'id' and 'name'
def parse_first_crew(row): 
    try:
        # Parse the JSON string from the 'crew' column
        crew_list = json.loads(row['crew'])
        if crew_list:
            # Return the first crew member with the job 'Director'
            director = next((member for member in crew_list if member['job'] == 'Director'), None)
            if director:  # Check if a director was found
                return (row['movie_id'], row['title'], director['id'], director['name'])
            else:
                return (row['movie_id'], row['title'], None, None)
        else: 
            return (row['movie_id'], row['title'], None, None)
    except (json.JSONDecodeError, IndexError): 
        return (row['movie_id'], row['title'], None, None)

# Apply the parsing functions to extract only the first cast and crew members
cast_info = credits_df.apply(parse_first_cast, axis=1)
crew_info = credits_df.apply(parse_first_crew, axis=1)

# Create separate DataFrames for cast and crew information
cast_df = pd.DataFrame(cast_info.tolist(), columns=['movie_id', 'title', 'cast_id', 'cast_name'])
crew_df = pd.DataFrame(crew_info.tolist(), columns=['movie_id', 'title', 'crew_id', 'crew_name'])

# Merge the cast and crew DataFrames on 'movie_id' and 'title'
merged_df = pd.merge(cast_df, crew_df, on=['movie_id', 'title'], how='outer')

# Drop any rows with missing data (if any) 
merged_df.dropna(inplace=True)

# Display the first few rows of the new DataFrame
print(merged_df.head())


   movie_id            title  cast_id      cast_name  crew_id        crew_name
0         5       Four Rooms   3129.0       Tim Roth   3110.0   Allison Anders
1        11        Star Wars      2.0    Mark Hamill      1.0     George Lucas
2        12     Finding Nemo     13.0  Albert Brooks      7.0   Andrew Stanton
3        13     Forrest Gump     31.0      Tom Hanks     24.0  Robert Zemeckis
4        14  American Beauty   1979.0   Kevin Spacey     39.0       Sam Mendes


jeffs version ( not run )

In [36]:
# Load the parquet file
file_path = 'data/credits_5000.parquet'
credits_df = pd.read_parquet(file_path)

# Function to parse the 'cast' column and extract only the first 'id' and 'name'
def parse_first_cast(row):
    try:
        # Parse the JSON string from the 'cast' column
        cast_list = json.loads(row['cast'])
        if cast_list:
            # Return the first cast member's id and name
            return (row['movie_id'], row['title'], cast_list[0]['id'], cast_list[0]['name'])
        else:
            return (row['movie_id'], row['title'], None, None)
    except (json.JSONDecodeError, IndexError):
        return (row['movie_id'], row['title'], None, None)

# Apply the parsing function to extract only the first cast member
first_cast_list = credits_df.apply(parse_first_cast, axis=1)

# Create a new DataFrame from the parsed cast list and rename it to 'credits_df'
credits_df = pd.DataFrame(first_cast_list.tolist(), columns=['movie_id', 'title', 'cast_id', 'cast_name'])

# Drop any rows with missing data (if any)
credits_df.dropna(inplace=True)

# Display the first few rows of the new DataFrame
print(credits_df.head())

   movie_id                                     title  cast_id  \
0     19995                                    Avatar  65731.0   
1       285  Pirates of the Caribbean: At World's End     85.0   
2    206647                                   Spectre   8784.0   
3     49026                     The Dark Knight Rises   3894.0   
4     49529                               John Carter  60900.0   

         cast_name  
0  Sam Worthington  
1      Johnny Depp  
2     Daniel Craig  
3   Christian Bale  
4    Taylor Kitsch  


In [37]:
print("Columns in merged_df:")
for col in merged_df.columns:
    print(col)

Columns in merged_df:
movie_id
title
cast_id
cast_name
crew_id
crew_name


In [38]:
print("Columns in movie_metadata_df:")
for col in movie_metadata_df.columns:
    print(col)

Columns in movie_metadata_df:
budget
id
original_language
popularity
production_countries
release_date
revenue
runtime
status
tagline
title
vote_average
vote_count
language
full_original_name
iso_3166_1
country_name
release_date_seasonholiday
holiday_name
Oscar Year
Year of Release
Award


In [39]:
import pandas as pd

# Sample DataFrames for demonstration
# Example:
# merged_df = pd.DataFrame({'movie_id': [1, 2, 3], 'some_other_data': ['A', 'B', 'C']})
# movies_metadata_df = pd.DataFrame({'id': [1, 2, 4], 'title': ['Movie 1', 'Movie 2', 'Movie 4']})

# Merge the DataFrames on the specified columns using an outer join
merged_data = pd.merge(
    movie_metadata_df, 
    merged_df, 
    left_on='id',         # Column from movies_metadata_df
    right_on='movie_id',  # Column from merged_df
    how='outer'           # Outer join to include all data
)

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_data.head())  # Show the first few rows of the merged DataFrame

# Save the merged DataFrame to a CSV file
output_file_path = 'merged_data.csv'
merged_data.to_csv(output_file_path, index=False)

print(f"Merged DataFrame saved to {output_file_path}")



Merged DataFrame:
        budget    id original_language popularity  \
0   $4,000,000   5.0                en       22.9   
1  $11,000,000  11.0                en      126.4   
2  $94,000,000  12.0                en       85.7   
3  $55,000,000  13.0                en      138.1   
4  $15,000,000  14.0                en       80.9   

                                production_countries release_date  \
0  [{"iso_3166_1": "US", "name": "United States o...   1995-12-09   
1  [{"iso_3166_1": "US", "name": "United States o...   1977-05-25   
2  [{"iso_3166_1": "US", "name": "United States o...   2003-05-30   
3  [{"iso_3166_1": "US", "name": "United States o...   1994-07-06   
4  [{"iso_3166_1": "US", "name": "United States o...   1999-09-15   

        revenue  runtime    status  \
0    $4,300,000     98.0  Released   
1  $775,398,007    121.0  Released   
2  $940,335,536    100.0  Released   
3  $677,945,399    142.0  Released   
4  $356,296,601    122.0  Released   

                   

In [40]:
# Print the merged DataFrame
print("Merged DataFrame:")
print(merged_data)  # Display the entire merged DataFram



Merged DataFrame:
           budget        id original_language popularity  \
0      $4,000,000       5.0                en       22.9   
1     $11,000,000      11.0                en      126.4   
2     $94,000,000      12.0                en       85.7   
3     $55,000,000      13.0                en      138.1   
4     $15,000,000      14.0                en       80.9   
...           ...       ...               ...        ...   
4798           $0  426067.0                en        0.0   
4799           $0  426469.0                en        0.7   
4800           $0  433715.0                en        0.0   
4801           $0  447027.0                en        0.0   
4802           $2  459488.0                en        0.1   

                                   production_countries release_date  \
0     [{"iso_3166_1": "US", "name": "United States o...   1995-12-09   
1     [{"iso_3166_1": "US", "name": "United States o...   1977-05-25   
2     [{"iso_3166_1": "US", "name": "United S

In [41]:
# Create a DataFrame to display the column names
columns_df = pd.DataFrame(merged_data.columns, columns=['Column Names'])

# Print the DataFrame in a table format
print("Columns in the merged DataFrame:")
print(columns_df)

Columns in the merged DataFrame:
                  Column Names
0                       budget
1                           id
2            original_language
3                   popularity
4         production_countries
5                 release_date
6                      revenue
7                      runtime
8                       status
9                      tagline
10                     title_x
11                vote_average
12                  vote_count
13                    language
14          full_original_name
15                  iso_3166_1
16                country_name
17  release_date_seasonholiday
18                holiday_name
19                  Oscar Year
20             Year of Release
21                       Award
22                    movie_id
23                     title_y
24                     cast_id
25                   cast_name
26                     crew_id
27                   crew_name


In [51]:
merged_data.head(5)

,budget,id,original_language,popularity,production_countries,release_date,revenue,runtime,status,tagline,...,holiday_name,Oscar Year,Year of Release,Award,movie_id,title_y,cast_id,cast_name,crew_id,crew_name
0,"$4,000,000",5.0,en,22.9,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,...,None,NaN,NaN,NaN,5.0,Four Rooms,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,en,126.4,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",...,None,1977,1977.0,Nominee,11.0,Star Wars,2.0,Mark Hamill,1.0,George Lucas
2,"$94,000,000",12.0,en,85.7,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2003-05-30,"$940,335,536",100.0,Released,"There are 3.7 trillion fish in the ocean, they...",...,None,NaN,NaN,NaN,12.0,Finding Nemo,13.0,Albert Brooks,7.0,Andrew Stanton
3,"$55,000,000",13.0,en,138.1,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1994-07-06,"$677,945,399",142.0,Released,"The world will never be the same, once you've ...",...,None,1994,1994.0,Winner,13.0,Forrest Gump,31.0,Tom Hanks,24.0,Robert Zemeckis
4,"$15,000,000",14.0,en,80.9,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-09-15,"$356,296,601",122.0,Released,Look closer.,...,None,1999,1999.0,Winner,14.0,American Beauty,1979.0,Kevin Spacey,39.0,Sam Mendes


In [47]:
cleaned_merged_df = merged_data.drop(columns=['original_language', 'production_countries', 'holiday_name', 'movie_id', 'title_y'])
cleaned_merged_df.head(2)

,budget,id,popularity,release_date,revenue,runtime,status,tagline,title_x,vote_average,...,iso_3166_1,country_name,release_date_seasonholiday,Oscar Year,Year of Release,Award,cast_id,cast_name,crew_id,crew_name
0,"$4,000,000",5.0,22.9,1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,NaN,NaN,NaN,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,126.4,1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977.0,Nominee,2.0,Mark Hamill,1.0,George Lucas


In [48]:
cleaned_merged_df = cleaned_merged_df.rename(columns={'id': 'movie_id', 'title_x': 'movie_title', 'iso_3166_1': 'country_short', 'release_date_seasonholiday': 'release_date_season', 'Oscar Year': 'oscar_year', 'Year of Release': 'year_of_release', 'Award': 'award', 'cast_id': 'actor_id', 'cast_name': 'actor', 'crew_id': 'director_id', 'crew_name': 'director_name'})
cleaned_merged_df.head(5)

,budget,movie_id,popularity,release_date,revenue,runtime,status,tagline,movie_title,vote_average,...,country_short,country_name,release_date_season,oscar_year,year_of_release,award,actor_id,actor,director_id,director_name
0,"$4,000,000",5.0,22.9,1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,NaN,NaN,NaN,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,126.4,1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977.0,Nominee,2.0,Mark Hamill,1.0,George Lucas
2,"$94,000,000",12.0,85.7,2003-05-30,"$940,335,536",100.0,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,...,US,United States of America,Spring,NaN,NaN,NaN,13.0,Albert Brooks,7.0,Andrew Stanton
3,"$55,000,000",13.0,138.1,1994-07-06,"$677,945,399",142.0,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,...,US,United States of America,Summer,1994,1994.0,Winner,31.0,Tom Hanks,24.0,Robert Zemeckis
4,"$15,000,000",14.0,80.9,1999-09-15,"$356,296,601",122.0,Released,Look closer.,American Beauty,7.9,...,US,United States of America,Fall,1999,1999.0,Winner,1979.0,Kevin Spacey,39.0,Sam Mendes


In [49]:
# Replace NaN values in the 'award' column with 'Non-nominee'
cleaned_merged_df['award'] = cleaned_merged_df['award'].fillna('Non-nominee')

# Display the updated DataFrame
cleaned_merged_df.head()

,budget,movie_id,popularity,release_date,revenue,runtime,status,tagline,movie_title,vote_average,...,country_short,country_name,release_date_season,oscar_year,year_of_release,award,actor_id,actor,director_id,director_name
0,"$4,000,000",5.0,22.9,1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,NaN,NaN,Non-nominee,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,126.4,1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977.0,Nominee,2.0,Mark Hamill,1.0,George Lucas
2,"$94,000,000",12.0,85.7,2003-05-30,"$940,335,536",100.0,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,...,US,United States of America,Spring,NaN,NaN,Non-nominee,13.0,Albert Brooks,7.0,Andrew Stanton
3,"$55,000,000",13.0,138.1,1994-07-06,"$677,945,399",142.0,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,...,US,United States of America,Summer,1994,1994.0,Winner,31.0,Tom Hanks,24.0,Robert Zemeckis
4,"$15,000,000",14.0,80.9,1999-09-15,"$356,296,601",122.0,Released,Look closer.,American Beauty,7.9,...,US,United States of America,Fall,1999,1999.0,Winner,1979.0,Kevin Spacey,39.0,Sam Mendes


In [50]:
# Replace NaN values in 'oscar_year' and 'year_of_release' columns with 0
cleaned_merged_df['oscar_year'] = cleaned_merged_df['oscar_year'].fillna(0)
cleaned_merged_df['year_of_release'] = cleaned_merged_df['year_of_release'].fillna(0)

# Display the updated DataFrame
cleaned_merged_df.head(5)

,budget,movie_id,popularity,release_date,revenue,runtime,status,tagline,movie_title,vote_average,...,country_short,country_name,release_date_season,oscar_year,year_of_release,award,actor_id,actor,director_id,director_name
0,"$4,000,000",5.0,22.9,1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,0,0.0,Non-nominee,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,126.4,1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977.0,Nominee,2.0,Mark Hamill,1.0,George Lucas
2,"$94,000,000",12.0,85.7,2003-05-30,"$940,335,536",100.0,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,...,US,United States of America,Spring,0,0.0,Non-nominee,13.0,Albert Brooks,7.0,Andrew Stanton
3,"$55,000,000",13.0,138.1,1994-07-06,"$677,945,399",142.0,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,...,US,United States of America,Summer,1994,1994.0,Winner,31.0,Tom Hanks,24.0,Robert Zemeckis
4,"$15,000,000",14.0,80.9,1999-09-15,"$356,296,601",122.0,Released,Look closer.,American Beauty,7.9,...,US,United States of America,Fall,1999,1999.0,Winner,1979.0,Kevin Spacey,39.0,Sam Mendes


In [52]:
print(cleaned_merged_df.dtypes)

budget                         object
movie_id                      float64
popularity                     object
release_date           datetime64[ns]
revenue                        object
runtime                       float64
status                         object
tagline                        object
movie_title                    object
vote_average                  float64
vote_count                     object
language                       object
full_original_name             object
country_short                  object
country_name                   object
release_date_season            object
oscar_year                     object
year_of_release               float64
award                          object
actor_id                      float64
actor                          object
director_id                   float64
director_name                  object
dtype: object


In [59]:
# Check if there are any NaN values in the DataFrame
cleaned_merged_df.isna().sum()

budget                   5
movie_id                 5
popularity               5
release_date             6
revenue                  5
runtime                  7
status                   5
tagline                843
movie_title              5
vote_average             5
vote_count               5
language                 5
full_original_name       5
country_short          176
country_name           176
release_date_season      6
oscar_year               0
year_of_release          0
award                    0
actor_id                49
actor                   49
director_id             49
director_name           49
dtype: int64

In [60]:
cleaned_merged_df = cleaned_merged_df.dropna()
cleaned_merged_df

,budget,movie_id,popularity,release_date,revenue,runtime,status,tagline,movie_title,vote_average,...,country_short,country_name,release_date_season,oscar_year,year_of_release,award,actor_id,actor,director_id,director_name
0,"$4,000,000",5.0,22.9,1995-12-09,"$4,300,000",98.0,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,0,0.0,Non-nominee,3129.0,Tim Roth,3110.0,Allison Anders
1,"$11,000,000",11.0,126.4,1977-05-25,"$775,398,007",121.0,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977.0,Nominee,2.0,Mark Hamill,1.0,George Lucas
2,"$94,000,000",12.0,85.7,2003-05-30,"$940,335,536",100.0,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,...,US,United States of America,Spring,0,0.0,Non-nominee,13.0,Albert Brooks,7.0,Andrew Stanton
3,"$55,000,000",13.0,138.1,1994-07-06,"$677,945,399",142.0,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,...,US,United States of America,Summer,1994,1994.0,Winner,31.0,Tom Hanks,24.0,Robert Zemeckis
4,"$15,000,000",14.0,80.9,1999-09-15,"$356,296,601",122.0,Released,Look closer.,American Beauty,7.9,...,US,United States of America,Fall,1999,1999.0,Winner,1979.0,Kevin Spacey,39.0,Sam Mendes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,"$8,000,000",389425.0,2.1,2016-05-06,$0,106.0,Released,There is no honor in revenge.,Code of Honor,4.1,...,US,United States of America,Spring,0,0.0,Non-nominee,23880.0,Steven Seagal,101233.0,Michael Winnick
4791,"$2,000,000",394047.0,1.2,2016-07-08,$0,101.0,Released,She ended up on the wrong side of right.,The Dog Lover,5.3,...,US,United States of America,Summer,0,0.0,Non-nominee,1440057.0,Allison Paige,474199.0,Alex Ranarivelo
4795,$0,407887.0,6.1,2016-07-27,$0,110.0,Released,The Odds Were 5000 to 1 … One was all They Nee...,Operation Chromite,5.8,...,KR,South Korea,Summer,0,0.0,Non-nominee,73249.0,Lee Jung-jae,78866.0,John H. Lee
4797,"$130,000,000",417859.0,20.7,2011-10-28,"$554,987,477",90.0,Released,Live for danger. Fight for justice. Pray for m...,Puss in Boots,6.4,...,US,United States of America,Fall,0,0.0,Non-nominee,3131.0,Antonio Banderas,12098.0,Chris Miller


In [65]:
# Function to convert shorthand notations like 'K' to actual integers
def convert_vote_count(value):
    if isinstance(value, str):
        value = value.upper()
        if 'K' in value:
            # Remove the 'K' and multiply by 1000
            return int(float(value.replace('K', '')) * 1000)
        else:
            # Handle other cases if needed (e.g., 'M' for millions)
            return int(value)
    return int(value)

# Apply the function to the 'vote_count' column
cleaned_merged_df['vote_count'] = cleaned_merged_df['vote_count'].apply(convert_vote_count)

# Verify the column after conversion
print(cleaned_merged_df['vote_count'].head())


0     500
1    6600
2    6100
3    7900
4    3300
Name: vote_count, dtype: int64


In [66]:
cleaned_merged_df = cleaned_merged_df.astype({
    'movie_id': 'int',
    'runtime': 'int',
    'vote_count': 'int',
    'year_of_release': 'int',
    'actor_id': 'int',
    'director_id': 'int'
})

print(cleaned_merged_df.dtypes)

budget                         object
movie_id                        int32
popularity                     object
release_date           datetime64[ns]
revenue                        object
runtime                         int32
status                         object
tagline                        object
movie_title                    object
vote_average                  float64
vote_count                      int32
language                       object
full_original_name             object
country_short                  object
country_name                   object
release_date_season            object
oscar_year                     object
year_of_release                 int32
award                          object
actor_id                        int32
actor                          object
director_id                     int32
director_name                  object
dtype: object


In [67]:
cleaned_merged_df

,budget,movie_id,popularity,release_date,revenue,runtime,status,tagline,movie_title,vote_average,...,country_short,country_name,release_date_season,oscar_year,year_of_release,award,actor_id,actor,director_id,director_name
0,"$4,000,000",5,22.9,1995-12-09,"$4,300,000",98,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,...,US,United States of America,Winter,0,0,Non-nominee,3129,Tim Roth,3110,Allison Anders
1,"$11,000,000",11,126.4,1977-05-25,"$775,398,007",121,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,...,US,United States of America,Spring,1977,1977,Nominee,2,Mark Hamill,1,George Lucas
2,"$94,000,000",12,85.7,2003-05-30,"$940,335,536",100,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,...,US,United States of America,Spring,0,0,Non-nominee,13,Albert Brooks,7,Andrew Stanton
3,"$55,000,000",13,138.1,1994-07-06,"$677,945,399",142,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,...,US,United States of America,Summer,1994,1994,Winner,31,Tom Hanks,24,Robert Zemeckis
4,"$15,000,000",14,80.9,1999-09-15,"$356,296,601",122,Released,Look closer.,American Beauty,7.9,...,US,United States of America,Fall,1999,1999,Winner,1979,Kevin Spacey,39,Sam Mendes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,"$8,000,000",389425,2.1,2016-05-06,$0,106,Released,There is no honor in revenge.,Code of Honor,4.1,...,US,United States of America,Spring,0,0,Non-nominee,23880,Steven Seagal,101233,Michael Winnick
4791,"$2,000,000",394047,1.2,2016-07-08,$0,101,Released,She ended up on the wrong side of right.,The Dog Lover,5.3,...,US,United States of America,Summer,0,0,Non-nominee,1440057,Allison Paige,474199,Alex Ranarivelo
4795,$0,407887,6.1,2016-07-27,$0,110,Released,The Odds Were 5000 to 1 … One was all They Nee...,Operation Chromite,5.8,...,KR,South Korea,Summer,0,0,Non-nominee,73249,Lee Jung-jae,78866,John H. Lee
4797,"$130,000,000",417859,20.7,2011-10-28,"$554,987,477",90,Released,Live for danger. Fight for justice. Pray for m...,Puss in Boots,6.4,...,US,United States of America,Fall,0,0,Non-nominee,3131,Antonio Banderas,12098,Chris Miller
